In [ ]:
import pandas as pd
import numpy as np
import os
import time
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
from tensorflow.keras.regularizers import Regularizer
# Figure out how to import regularizers
import tensorflow.keras.utils as ku 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/data/

/content/drive/MyDrive/data


In [ ]:
#The dataset (P.S. Look, it is TSV)
data=pd.read_csv('data_angry.csv')

# removing null values to avoid errors 
data.dropna(inplace = True) 
  
# converting to string data type
data["Title"]= data["Title"].astype(str)
  
# slicing till 2nd last element
data["lyric"]= data["Title"].str.slice(10,)
  
# display
data.head()

# data.album.drop_duplicates().tolist()
data.head()

,Title,lyric
0,[00:00.29]I took my lover to the sea,I took my lover to the sea
1,[00:05.01]Drowned him precious bride to be,Drowned him precious bride to be
2,[00:12.48]Bleeding heaven from the start,Bleeding heaven from the start
3,[00:16.82]Woe is rapture ripped apart,Woe is rapture ripped apart
4,"[00:21.27]Hey you're honest, aren't you?","Hey you're honest, aren't you?"


In [ ]:
data.lyric.drop_duplicates().tolist()

['I took my lover to the sea',
 'Drowned him precious bride to be',
 'Bleeding heaven from the start',
 'Woe is rapture ripped apart',
 "Hey you're honest, aren't you?",
 'Show me rapture torn apart',
 "Everybody's riddled with disease",
 'But I bet you say, "not me"',
 "So honest, aren't you?",
 "You're so honest, aren't you so?",
 '.',
 "My, aren't you",
 'Surely the finest of the brigade?',
 'Always right a portrait of dignity?',
 'Innocent and never suspectable',
 'Wait',
 'You smell like sh*t, not the truth',
 'Full of device, not devotion',
 'Conscience came right up to you',
 'And then you threw it back',
 'You are the scum of the earth',
 'You are the scum of the ocean',
 "To you it's above as below",
 'You smear your filth across the world',
 '',
 'In my hour of need',
 'Ha you’re not there',
 'And though I reached out for you',
 'Wouldn’t lend a hand',
 'Through the darkest hour',
 'Grace did not shine on me',
 'It feels so cold， very cold',
 'No one cares for me',
 'Did you 

In [ ]:
data.shape

(5767, 2)

In [ ]:
IND = data.lyric.drop_duplicates().tolist()
#Move in the index range
CHORUSES=[]
for i in IND:
    CHORUSES.append(data.lyric.tolist())

In [ ]:
CHORUSES=np.array(data.lyric.tolist())
CHORUSES=CHORUSES.tolist()
#Write everything in a single line 
TERM=''
for c in range(len(CHORUSES)):
    
    TERM= TERM+ CHORUSES[c]+ ' \n '

In [ ]:
TERM

'I took my lover to the sea \n Drowned him precious bride to be \n Bleeding heaven from the start \n Woe is rapture ripped apart \n Hey you\'re honest, aren\'t you? \n Show me rapture torn apart \n Everybody\'s riddled with disease \n But I bet you say, "not me" \n So honest, aren\'t you? \n You\'re so honest, aren\'t you so? \n . \n My, aren\'t you \n Surely the finest of the brigade? \n My, aren\'t you \n Always right a portrait of dignity? \n My, aren\'t you \n Innocent and never suspectable \n My, aren\'t you \n . \n Wait \n You smell like sh*t, not the truth \n Full of device, not devotion \n Conscience came right up to you \n And then you threw it back \n You are the scum of the earth \n You are the scum of the ocean \n To you it\'s above as below \n You smear your filth across the world \n You smear your filth across the world \n You smear your filth across the world \n You smear your filth across the world \n . \n  \n In my hour of need \n Ha you’re not there \n And though I re

In [ ]:
print("The unique characters:",sorted(set(TERM)))

The unique characters: ['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '~', '§', '‘', '’', '…', '′', '≡', '了', '人', '作', '制', '卐', '我', '朝', '王', '语', '＇', '（', '）', '，']


In [ ]:
to_remove = ['[', '\\', ']', '_', '{', '}','~', '§', '…','≡', '了', '人', '作', '制', '卐', '我', '朝', '王', '语', '\ufeff','(', ')','*', '（', '）', '‘', '’', '′', '＇', "'", "'", '.']
for symbol in to_remove:
    TERM = TERM.replace(symbol,"")

print("The unique characters:",sorted(set(TERM)))

The unique characters: ['\n', ' ', '!', '"', '&', ',', '-', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '，']


In [ ]:
#Write it into a file
f = open("choruses.txt", "a")
f.write(TERM)
f.close()

In [ ]:
text = open('choruses.txt', 'rb').read().decode(encoding='utf-8')

In [ ]:
text[0:5000]

"Another day wasted out of time \n I can't get out of this \n Altered state of mind \n I'm going overboard \n My conscience meets decline \n Into reality \n I know this can't be fine \n Cause I'm all messed up \n Making prefect nonsense \n Drowning in my doubt too well \n Cause I'm all messed up \n Going nowhere fast but circles in my mind \n So blind \n Who are these voices in my head \n I can't go on like this \n Living like the dead \n I haven't slept so long \n Feeling sad I dread \n I'm talking to myself \n Forgot what I just said \n Cause I'm all messed up \n Making prefect nonsense \n Drowning in my doubt too well \n Cause I'm all messed up \n Going nowhere fast but circles in my mind \n So blind \n Well I hold my only enemy as closely as a friend \n And I sold my own reality to further my descent \n Self-destruction taking over it's so easy to pretend \n Introduction to this nightmare may never end \n Can anyone help me drag my heels \n I'm running overtime \n I can't hold down

In [ ]:
# Splitting the string into sentences, while converting whole data into lowercase.
corpus = text.lower().split("\n")
# To make sure no sentence appears twice in our corpus, we use set. Otherwise, it will make the model biased.
corpus = list(set(corpus))
print(corpus)

[' now iras folks were hungry ', ' and i wont kill you ', ' my words are like a dagger with a jagged edge ', ' you don’t know a thing about me ', ' and the way things seem, i shouldnt have to pay these shrinks ', " it drives me, yes it drives me  damn you're so hot  ", ' i cant buy into what youre selling ', ' dont try to understand， knowing you im probably wrong ', ' plus dre told me stay out, this just wasn t my beef ', ' of his spoken word ', ' it wouldnt pain me more to bury you rich ', ' i picked him and things went so bad ', ' i put on my mothers make up, get naked and run around ', ' a plague and platinum status is wack if im not the baddest so ', ' cuz ive got nothing left to prove, cmon ', ' nothings gunna change ', ' theyre gonna wanna know whats up! ', ' butall i really want to know is ', ' you know i just don t get it ', " dumb fucking question, don't let yourself fall asleep ", ' im full of hate, dont fuck with me ', ' em aight ', ' why would i wanna destroy something i he

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [ ]:
tokenizer.word_index

{'i': 1,
 'you': 2,
 'the': 3,
 'to': 4,
 'and': 5,
 'a': 6,
 'it': 7,
 'me': 8,
 'my': 9,
 'in': 10,
 'of': 11,
 'that': 12,
 'all': 13,
 't': 14,
 'your': 15,
 'but': 16,
 'just': 17,
 'be': 18,
 's': 19,
 'what': 20,
 'on': 21,
 'is': 22,
 'so': 23,
 'this': 24,
 'know': 25,
 'up': 26,
 'im': 27,
 'like': 28,
 'for': 29,
 'with': 30,
 'm': 31,
 'out': 32,
 "i'm": 33,
 'now': 34,
 'not': 35,
 'never': 36,
 'when': 37,
 'we': 38,
 'can': 39,
 'one': 40,
 'no': 41,
 'at': 42,
 'he': 43,
 'do': 44,
 'dont': 45,
 'don': 46,
 'they': 47,
 'if': 48,
 'back': 49,
 'get': 50,
 "don't": 51,
 'think': 52,
 'its': 53,
 'down': 54,
 'see': 55,
 'need': 56,
 'cause': 57,
 'take': 58,
 'as': 59,
 'too': 60,
 're': 61,
 'time': 62,
 'gonna': 63,
 'say': 64,
 'shit': 65,
 "it's": 66,
 'from': 67,
 'was': 68,
 'fuck': 69,
 'go': 70,
 'how': 71,
 'youre': 72,
 'why': 73,
 'there': 74,
 'want': 75,
 'have': 76,
 "you're": 77,
 'his': 78,
 'got': 79,
 'way': 80,
 'let': 81,
 'will': 82,
 'wanna': 83,
 '

In [ ]:
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
input_sequences

[[34, 2256],
 [34, 2256, 1520],
 [34, 2256, 1520, 143],
 [34, 2256, 1520, 143, 592],
 [5, 1],
 [5, 1, 273],
 [5, 1, 273, 128],
 [5, 1, 273, 128, 2],
 [9, 286],
 [9, 286, 91],
 [9, 286, 91, 28],
 [9, 286, 91, 28, 6],
 [9, 286, 91, 28, 6, 2642],
 [9, 286, 91, 28, 6, 2642, 30],
 [9, 286, 91, 28, 6, 2642, 30, 6],
 [9, 286, 91, 28, 6, 2642, 30, 6, 2643],
 [9, 286, 91, 28, 6, 2642, 30, 6, 2643, 2644],
 [2, 711],
 [2, 711, 25],
 [2, 711, 25, 6],
 [2, 711, 25, 6, 129],
 [2, 711, 25, 6, 129, 96],
 [2, 711, 25, 6, 129, 96, 8],
 [5, 3],
 [5, 3, 80],
 [5, 3, 80, 158],
 [5, 3, 80, 158, 545],
 [5, 3, 80, 158, 545, 1],
 [5, 3, 80, 158, 545, 1, 783],
 [5, 3, 80, 158, 545, 1, 783, 76],
 [5, 3, 80, 158, 545, 1, 783, 76, 4],
 [5, 3, 80, 158, 545, 1, 783, 76, 4, 712],
 [5, 3, 80, 158, 545, 1, 783, 76, 4, 712, 106],
 [5, 3, 80, 158, 545, 1, 783, 76, 4, 712, 106, 1521],
 [7, 593],
 [7, 593, 8],
 [7, 593, 8, 274],
 [7, 593, 8, 274, 7],
 [7, 593, 8, 274, 7, 593],
 [7, 593, 8, 274, 7, 593, 8],
 [7, 593, 8, 274

In [ ]:
corpus[0]

' now iras folks were hungry '

In [ ]:
tokenizer.word_index["still"]

140

In [ ]:
# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,
                       maxlen = max_sequence_len, padding='pre'))

In [ ]:
input_sequences

array([[   0,    0,    0, ...,    0,   34, 2256],
       [   0,    0,    0, ...,   34, 2256, 1520],
       [   0,    0,    0, ..., 2256, 1520,  143],
       ...,
       [   0,    0,    0, ...,    0,  449,    5],
       [   0,    0,    0, ...,  449,    5, 2024],
       [   0,    0,    0, ...,    5, 2024,  224]], dtype=int32)

In [ ]:
input_sequences.shape

(44367, 21)

In [ ]:
tokenizer.word_index['universe']

2743

In [ ]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
label.shape

(44367, 4215)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))  #(# Your Embedding Layer)
model.add(Bidirectional(LSTM(150, return_sequences=True)))  #(# An LSTM Layer)
model.add(Dropout(0.2))  #(# A dropout layer)
model.add(LSTM(100))  #(# Another LSTM Layer)
model.add(Dense(total_words/2, activation='relu'))  #(# A Dense Layer including regularizers)
model.add(Dense(total_words, activation='softmax'))  #(# A Dense Layer)
# Pick an optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')  #(# Pick a loss function and an optimizer)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 100)           421500    
                                                                 
 bidirectional (Bidirectiona  (None, 20, 300)          301200    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 20, 300)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 2107)              212807    
                                                                 
 dense_1 (Dense)             (None, 4215)              8885220   
                                                        

In [ ]:
model.fit(predictors, label, epochs= 50, verbose=1)

Epoch 1/50
1387/1387 [==============================] - 44s 24ms/step - loss: 6.3829 - accuracy: 0.0382
Epoch 2/50
1387/1387 [==============================] - 20s 15ms/step - loss: 5.7256 - accuracy: 0.0752
Epoch 3/50
1387/1387 [==============================] - 18s 13ms/step - loss: 5.3157 - accuracy: 0.1012
Epoch 4/50
1387/1387 [==============================] - 17s 12ms/step - loss: 4.9760 - accuracy: 0.1188
Epoch 5/50
1387/1387 [==============================] - 19s 14ms/step - loss: 4.6676 - accuracy: 0.1404
Epoch 6/50
1387/1387 [==============================] - 17s 12ms/step - loss: 4.3724 - accuracy: 0.1586
Epoch 7/50
1387/1387 [==============================] - 23s 17ms/step - loss: 4.0663 - accuracy: 0.1854
Epoch 8/50
1387/1387 [==============================] - 19s 14ms/step - loss: 3.7469 - accuracy: 0.2210
Epoch 9/50
1387/1387 [==============================] - 23s 17ms/step - loss: 3.4181 - accuracy: 0.2668
Epoch 10/50
1387/1387 [==============================] - 17s 13m

In [ ]:
from keras.models import load_model


In [ ]:
model.save('Angry_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
# model = load_model('my_model.h5')

In [ ]:
def make_lyrics(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
        predicted = model.predict(token_list, verbose=0)
        classes = np.argmax(predicted,axis=1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == classes:
                output_word = word
                break
        seed_text += " " + output_word
    print(str(seed_text))

yhat_classes = model.predict_classes(X_test)

predict_x=model.predict(X_test) 

classes_x=np.argmax(predict_x,axis=1)

model.predict(x_test) > 0.5).astype("int32")

In [ ]:
make_lyrics("shotgun", 10)